In [1]:
import ROOT
from rootpy.tree import Tree, TreeModel, FloatCol, IntCol
from rootpy.io import root_open
from ROOT import gROOT, TCanvas, TF1, TFile, TTree, gRandom, TH1F

from ROOT import RooRealVar, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooGaussian,RooExponential, \
                 RooBinning, kRed, kBlue, kDotted,TString,RooAbsData, RooPlot, TCut, RooAbsData

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os, sys, time, random

from ROOT import TTree, TFile

# from root_numpy import root2array, rec2array, array2root

import pandas as pd
import numpy as np
import scipy 
import root_pandas as rp
import root_numpy as ry 

import pandas.core.common as com
from pandas.core.index import Index
from pandas.tools import plotting
from pandas.tools.plotting import scatter_matrix

from tqdm import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score

sys.path.append('/home/chasenberg/repos/')
sys.path.append('/home/chasenberg/repos/dopy')
from dopy import * 
from dopy.dolearn.sklearn_utils import plot_roc_curve, plot_classifier_output, plot_correlations
from dopy.dolearn.sklearn_utils import plot_feature_importances, plot_classifier_output, classify_unseen_data
#from dopy.sklearn_utils import plot_bdt_vars
from dopy.doplot.plotting import Plotter, Plot
from dopy.doanalysis.df_utils import add_min_max, add_eta 

/opt/rh/miniconda/envs/py3root6/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Create new ROOT File with relevant branches

In [8]:
#directories and files 
mc_dir_test = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks'
mc_file_test ='Bd2JpsiKS_after_stripping_flat.root'
mc_dir_test = os.path.join(mc_dir_test, mc_file_test)

In [9]:
cut_string = 'B0_FitDaughtersConst_status==0&B0_FitPVConst_status==0&B0_BKGCAT==0&(B0_L0MuonDecision_Dec==1|B0_L0DiMuonDecision_Dec==1|B0_L0MuonHighDecision_Dec==1)&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)&(B0_Hlt1DiMuonHighMassDecision_Dec==1|B0_Hlt1TrackMVADecision_Dec==1)&(B0_Hlt2DiMuonJPsiDecision_Dec==1|B0_Hlt2DiMuonDetachedJPsiDecision_Dec==1)'               
tree_mc = 'Bd2JpsiKs'

In [10]:
variables_mc = [
'B0_M',
'idxPV',
'piplus_TRACK_Type',
'eventNumber',
'B0_FitDaughtersConst_M',
'runNumber'
]

''''B0_FitPVConst_status',
'B0_FitDaughtersConst_KS0_P1_PT', 
'B0_FitDaughtersConst_KS0_P0_PT',
'B0_FitDaughtersConst_KS0_P0_IPCHI2', 
'B0_FitDaughtersConst_KS0_P1_IPCHI2',
'B0_FitDaughtersConst_J_psi_1S_P0_PT', 
'B0_FitDaughtersConst_J_psi_1S_P1_PT',
'B0_FitPVConst_KS0_tau',
'B0_FitPVConst_KS0_tauErr',
'B0_BKGCAT',
'B0_FitPVConst_IPCHI2',
'B0_FitPVConst_IPCHI2_flat',
'B0_FitPVConst_MinIPCHI2anyPV_flat',
'B0_FitDaughtersConst_M',
'B0_TAU',
'B0_TAUERR',
'B0_TRUETAU',
'B0_FitDaughtersConst_status_flat',
'B0_FitPVConst_status_flat',
'piplus_TRACK_Type',
'piminus_MINIPCHI2',
'piplus_MINIPCHI2',
'muminus_MINIPCHI2',
'muplus_MINIPCHI2',
'piplus_ProbNNp',
'piminus_ProbNNp',
'eventNumber',
'runNumber',
'B0_FitwithoutConst_chi2',
'B0_FitwithoutConst_nDOF',
'B0_FitPVConst_chi2',
'B0_FitPVConst_nDOF'''

"'B0_FitPVConst_status',\n'B0_FitDaughtersConst_KS0_P1_PT', \n'B0_FitDaughtersConst_KS0_P0_PT',\n'B0_FitDaughtersConst_KS0_P0_IPCHI2', \n'B0_FitDaughtersConst_KS0_P1_IPCHI2',\n'B0_FitDaughtersConst_J_psi_1S_P0_PT', \n'B0_FitDaughtersConst_J_psi_1S_P1_PT',\n'B0_FitPVConst_KS0_tau',\n'B0_FitPVConst_KS0_tauErr',\n'B0_BKGCAT',\n'B0_FitPVConst_IPCHI2',\n'B0_FitPVConst_IPCHI2_flat',\n'B0_FitPVConst_MinIPCHI2anyPV_flat',\n'B0_FitDaughtersConst_M',\n'B0_TAU',\n'B0_TAUERR',\n'B0_TRUETAU',\n'B0_FitDaughtersConst_status_flat',\n'B0_FitPVConst_status_flat',\n'piplus_TRACK_Type',\n'piminus_MINIPCHI2',\n'piplus_MINIPCHI2',\n'muminus_MINIPCHI2',\n'muplus_MINIPCHI2',\n'piplus_ProbNNp',\n'piminus_ProbNNp',\n'eventNumber',\n'runNumber',\n'B0_FitwithoutConst_chi2',\n'B0_FitwithoutConst_nDOF',\n'B0_FitPVConst_chi2',\n'B0_FitPVConst_nDOF"

In [11]:
#variables_mc = ['B0_M']

In [12]:
#read root file
signal_dataframe_test = rp.read_root(mc_dir_test,key=tree_mc,where=cut_string,columns=variables_mc, flatten=False)
signal_dataframe_test = signal_dataframe_test.replace([np.inf, -np.inf], np.nan)
signal_dataframe_test = signal_dataframe_test.dropna()
signal_dataframe_test.to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/test.root',key='test')

/opt/rh/miniconda/envs/py3root6/lib/python3.4/site-packages/root_numpy/_tree.py:570: UserWarning: converter for dtype('O') is not implemented (skipping)
  _librootnumpy.array2root(arr, filename, treename, mode)


In [21]:
signal_dataframe_test.shape[0]

565715

In [15]:
signal_dataframe_test.shape[0]

1044051

### Efficiency part

In [ ]:
#directories and files 
mc_dir = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/wrong_pv'
mc_file ='Bd2JpsimumuKS_mc_2015_2016_selected.root'
mc_dir = os.path.join(mc_dir, mc_file)

In [ ]:
#criteria and information for read in data
#branch_names_mc = ['B0_M','Delta_TAU','']
cut_string_mc = ''
tree_mc = 'Bd2JpsiKs'

In [ ]:
#read root file
signal_dataframe = rp.read_root(mc_dir,key=tree_mc,where=cut_string_mc, flatten=False)
signal_dataframe = signal_dataframe.replace([np.inf, -np.inf], np.nan)
signal_dataframe = signal_dataframe.dropna()

In [ ]:
'''signal_dataframe[signal_dataframe['BDTresponse']>0.62]['Delta_TAU']
p = Plotter('/home/chasenberg/plots/selection/')
p.create_plots(signal_dataframe,'Delta_TAU',component_labels=['MC'])
p['Delta_TAU'].set_range(-0.001,0.001)
#bdt_plot.plot()
p.plot()

'''

Check efficiencies of all selection steps
=================================================

In [ ]:
test = signal_dataframe.query('idxPV==0')

BDT cut of 0.48
------------------------

In [ ]:
bdt_cut = 'BDTresponse>0.48'
efficiency = test.query(bdt_cut).shape[0]/test.shape[0]
print(efficiency)

Apply the L0 veto
--------------------------

In [ ]:
l0_cut = '&((abs(1115.683-varLambda0MassHypo_ppluspiminus)>10)|piplus_ProbNNp<0.05)&((abs(1115.683-varLambda0MassHypo_pminuspiplus)>10)|(piminus_ProbNNp<0.05))'                   

In [ ]:
l0_cutefficiency = test.query(bdt_cut+l0_cut).shape[0]/test.query(bdt_cut).shape[0]
print(l0_cutefficiency)

Apply the MinIPCHI2 cut > 6
--------------------------

In [ ]:
pv_cut = "&B0_FitPVConst_MinIPCHI2anyPV_flat>6"
test_2 = test.query(bdt_cut+l0_cut+pv_cut)

In [ ]:
l0_cutefficiency = test.query(bdt_cut+l0_cut+pv_cut).shape[0]/test.query(bdt_cut).shape[0]
print(l0_cutefficiency)

In [ ]:
test_2.to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/wrong_pv/final/minip_cut.root',key='Bd2JpsiKS')

Efficiency of random selection
-------------------------------

In [ ]:
test_3 = rp.read_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/wrong_pv/final/test/test.root',key='test')

In [ ]:
eff = test_3.shape[0]/test.query(bdt_cut+l0_cut).shape[0]
print(eff)